In [24]:
def generate(numRows: int):
        ret = [[1]]
        if numRows == 1:
            return ret
        ret = [[1],[1,1]]
        if numRows == 2:
            return ret
        for row in range(3,numRows+1):
            arr = [0]*row
            arr[0] = 1
            arr[row-1] = 1
            for idx in range(1,row-1):
                arr[idx] = ret[row-2][idx-1] + ret[row-2][idx]
            ret.append(arr)
        return ret

In [25]:
generate(3)

[[1], [1, 1], [1, 2, 1]]

In [32]:
import pandas as pd
df = {}

for x in range(8):
    df[x] = pd.read_json(f'./MyData/StreamingHistory{x}.json')

In [38]:
df2 = pd.concat([df[k] for k in df])

In [39]:
df2

,endTime,artistName,trackName,msPlayed
0,2020-06-24 21:49,Fleetwood Mac,Dreams - 2004 Remaster,12956
1,2020-08-02 15:10,Post Animal,When I Get Home,323200
2,2020-08-02 15:14,Post Animal,Ralphie,275933
3,2020-08-02 15:14,Post Animal,Goggles,5698
4,2020-08-02 15:19,Japanese Breakfast,Machinist,10797
...,...,...,...,...
4250,2021-07-06 23:12,The Cardigans,Lovefool,193893
4251,2021-07-06 23:28,Bon Entendeur,"L'amour, l'amour, l'amour",19587
4252,2021-07-06 23:48,LAUSSE THE CAT,Redstripe Rhapsody,121835
4253,2021-07-06 23:52,"Tyler, The Creator",SWEET / I THOUGHT YOU WANTED TO DANCE (feat. B...,3784


In [42]:
dfs = df2.groupby(['artistName','trackName']).size().reset_index().groupby('trackName')[[0]].max().sort_values(by=[0],ascending=False)

In [43]:
dfs.head(50)

,0
trackName,
Locket,469
Guilty Conscience,446
Anomalie bleue,366
Every Single Thing,357
Space Song,351
Someday,331
Forever (feat. The Hacker & Electric Youth),289
Movement 6,261
Instant Crush (feat. Julian Casablancas),225


In [28]:
pd.set_option('display.max_rows', 50)

In [46]:
import requests, json, time, pandas as pd
sys.path.append("..")
import os
from dotenv import load_dotenv
load_dotenv()

key = os.getenv(key="LASTFM_KEY")
username = "ezfire"
pause_duration = .2

def get_scrobbles(method='recenttracks', username=username, key=key, limit=200, extended=0, page=1, pages=0):
    '''
    method: api method
    username/key: api credentials
    limit: api lets you retrieve up to 200 records per call
    extended: api lets you retrieve extended data for each track, 0=no, 1=yes
    page: page of results to start retrieving at
    pages: how many pages of results to retrieve. if 0, get as many as api can return.
    '''
    # initialize url and lists to contain response fields
    url = 'https://ws.audioscrobbler.com/2.0/?method=user.get{}&user={}&api_key={}&limit={}&extended={}&page={}&format=json'
    responses = []
    artist_names = []
    artist_mbids = []
    album_names = []
    album_mbids = []
    track_names = []
    track_mbids = []
    timestamps = []
    
    # make first request, just to get the total number of pages
    request_url = url.format(method, username, key, limit, extended, page)
    response = requests.get(request_url).json()
    total_pages = int(response[method]['@attr']['totalPages'])
    if pages > 0:
        total_pages = min([total_pages, pages])
        
    print('{} total pages to retrieve'.format(total_pages))
    
    # request each page of data one at a time
    for page in range(1, int(total_pages) + 1, 1):
        if page % 10 == 0: print(page, end=' ')
        time.sleep(pause_duration)
        request_url = url.format(method, username, key, limit, extended, page)
        responses.append(requests.get(request_url))
    
    # parse the fields out of each scrobble in each page (aka response) of scrobbles
    for response in responses:
        scrobbles = response.json()
        for scrobble in scrobbles[method]['track']:
            # only retain completed scrobbles (aka, with timestamp and not 'now playing')
            if 'date' in scrobble.keys():
                artist_names.append(scrobble['artist']['#text'])
                artist_mbids.append(scrobble['artist']['mbid'])
                album_names.append(scrobble['album']['#text'])
                album_mbids.append(scrobble['album']['mbid'])
                track_names.append(scrobble['name'])
                track_mbids.append(scrobble['mbid'])
                timestamps.append(scrobble['date']['uts'])
                
    # create and populate a dataframe to contain the data
    df = pd.DataFrame()
    df['artist'] = artist_names
    df['artist_mbid'] = artist_mbids
    df['album'] = album_names
    df['album_mbid'] = album_mbids
    df['track'] = track_names
    df['track_mbid'] = track_mbids
    df['timestamp'] = timestamps
    df['datetime'] = pd.to_datetime(df['timestamp'].astype(int), unit='s')
    
    return df

In [51]:
# get all scrobbled tracks ever, in order of recency (pages=0 to get all)
scrobbles = get_scrobbles(pages=0)

25 total pages to retrieve
10 20 

In [52]:
# save the dataset
scrobbles.to_csv('lastfm_top.csv', index=None, encoding='utf-8')
print('{:,} total rows'.format(len(scrobbles)))
scrobbles.head(-10)

0 total rows


,artist,artist_mbid,album,album_mbid,track,track_mbid,timestamp,datetime


In [54]:
pause_duration = .2
url = 'https://ws.audioscrobbler.com/2.0/?method=user.get{}&user={}&api_key={}&limit={}&extended={}&page={}&format=json'
limit = 200 #api lets you retrieve up to 200 records per call
extended = 0 #api lets you retrieve extended data for each track, 0=no, 1=yes
page = 1 #page of results to start retrieving at
method = 'toptracks'
request_url = url.format(method, username, key, limit, extended, page)
artist_names = []
track_names = []
play_counts = []
response = requests.get(request_url).json()
for item in response[method]['track']:
    artist_names.append(item['artist']['name'])
    track_names.append(item['name'])
    play_counts.append(item['playcount'])

top_tracks = pd.DataFrame()
top_tracks['artist'] = artist_names
top_tracks['track'] = track_names
top_tracks['play_count'] = play_counts
top_tracks.to_csv('lastfm_top_tracks.csv', index=None, encoding='utf-8')


,artist,track,play_count
0,Crumb,Locket,283
1,070 Shake,Guilty Conscience,265
2,HOMESHAKE,Every Single Thing,254
3,L'Impératrice,Anomalie bleue,216
4,The Strokes,Someday,203


In [57]:
method = 'topartists'
request_url = url.format(method, username, key, limit, extended, page)
artist_names = []
play_counts = []
response = requests.get(request_url).json()
for item in response[method]['artist']:
    artist_names.append(item['name'])
    play_counts.append(item['playcount'])

top_artists = pd.DataFrame()
top_artists['artist'] = artist_names
top_artists['play_count'] = play_counts
top_artists.to_csv('lastfm_top_artists.csv', index=None, encoding='utf-8')


,artist,play_count
0,Kanye West,1197
1,The Weeknd,967
2,The Strokes,957
3,L'Impératrice,788
4,"Tyler, the Creator",698


In [61]:
top_artists

,artist,play_count
0,Kanye West,1197
1,The Weeknd,967
2,The Strokes,957
3,L'Impératrice,788
4,"Tyler, the Creator",698
...,...,...
195,Sports,36
196,Yheti,36
197,Demuja,35
198,Isaiah Rashad,35


In [60]:
method = 'topalbums'
request_url = url.format(method, username, key, limit, extended, page)
artist_names = []
album_names = []
play_counts = []
response = requests.get(request_url).json()
for item in response[method]['album']:
    artist_names.append(item['artist']['name'])
    album_names.append(item['name'])
    play_counts.append(item['playcount'])

top_albums = pd.DataFrame()
top_albums['artist'] = artist_names
top_albums['album'] = album_names
top_albums['play_count'] = play_counts
top_albums.to_csv('lastfm_top_albums.csv', index=None, encoding='utf-8')
top_albums

,artist,album,play_count
0,The Strokes,Is This It,582
1,The Weeknd,Starboy,350
2,Kanye West,The Life of Pablo,313
3,Crumb,Locket,299
4,HOMESHAKE,Fresh Air,267
...,...,...,...
195,Crumb,Ice Melt,39
196,Cuco,Amor de Siempre (Mariachi Version),39
197,Disclosure,Where Angels Fear To Tread,39
198,Floating Points,Bias,39


In [33]:
import pandas as pd

x = pd.read_csv('lastfm_scrobbles.csv',parse_dates=['datetime'])

z = x[['track','datetime']]

In [38]:
z.groupby(['track']).count().sort_values(by="datetime")

,datetime
track,
🌺 Unreleased Ye,1
Hollow Be They Squad,1
Hollow Point Bullets Recalled That Don't Explode In Targets,1
September,1
Holy (feat. Chance the Rapper),1
...,...
Someday,207
Anomalie bleue,217
Every Single Thing,254


In [53]:
z['count'] = 1
z['datetime'] = z['datetime'] - pd.to_timedelta(7, unit='d')
a = z.groupby(['track', pd.Grouper(key='datetime', freq='W-MON')])['count'].sum().reset_index().sort_values('count')
b = z.groupby(['track', pd.Grouper(key='datetime', freq='D')])['count'].sum().reset_index().sort_values('count')
c = z.groupby(['track', pd.Grouper(key='datetime', freq='M')])['count'].sum().reset_index().sort_values('count')

/home/ezfire/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/ezfire/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [60]:
c.sort_values('count',ascending=False).head(20)

,track,datetime,count
6142,Never In Your Sun,2021-05-31,97
2855,Every Single Thing,2021-04-30,86
3721,Guilty Conscience,2021-04-30,85
3828,Hard to Explain,2021-05-31,80
2356,Disco,2021-05-31,80
10605,abusive,2021-04-30,80
706,Anomalie bleue,2021-04-30,79
8809,TELL EM,2021-04-30,75
8396,Something About Us,2021-04-30,71
11043,what was the last thing u said,2021-04-30,70
